# Scissor Jack Failure Modes

In [12]:
using Roots
using ForwardDiff

In [13]:
#############################################################################
# Declaration of commonly used variables:
#############################################################################

# Diameter of pin and holes in diagonal member and crossbar:
d = 0.25 #inches

# Thickness of diagonal member:
t_d = 0.125 #inches

# Thickness of crossbar: assumes square hollow crossbar
t_cb = 0.06 #inches

#OR

#Diameter of crossbar: assumes crossbar is solid round
d_cb = 0.5 #inches

# Length from center of hole to edge of bar (both diagonal bar and crossbar):
l = 0.4 #inches

# Width of diagonal member:
w_d = 1.25 #inches

# Width of crossbar: assumes square hollow crossbar
w_cb = 0.5 #inches

# Number of shear planes on pin:
N = 2

# Length of the crossbar:
l_cb = 13.02 #inches

# Length of the diagonal member:
l_d = 8.5 #inches

# Yield strength of the diagonal member:
S_y_d = 16000 #psi

# Yield strength of the crossbar:
S_y_cb = 32000 #psi

# Yield strength of the pin: (This is actually the tensile strength...)
S_y_p = 140000 #psi

# Modulus of Elasticity of the crossbar:
E_cb = 30000000 #psi

# End-conditiion Constant C:
C = 1

# Applied Force:
F_A = 700 #lbf

#############################################################################
# Declaration of commonly used functions:
#############################################################################

"""
scissor_forces(applied_force, cross_bar_length; diagonal_length = 8.5)
This function returns the force in the diagonal and cross-bar members of a scissor jack.

applied_force is the tension force applied to the jack in Newtons
cross_bar_length is the length of the cross bar in inches
diagonal_length is the length of the diagonal bar in inches. Its default length is 8.5 inches.
"""
function scissor_forces(applied_force, cross_bar_length; diagonal_length = 8.5)
    alpha = acos(cross_bar_length / (2 * diagonal_length)) #rad
    F_d = (1/2) * (applied_force / sin(alpha)) #lbf
    F_cb = applied_force * cot(alpha) #lbf
    return F_d, F_cb
end

"""
diagonal_bar_bearing_stress(force, diameter, thickness)
This function returns the bearing stress in the diagonal bar

force is the force in the diagonal member
diameter is the diameter of the hole in the diagonal member
thickness is the thickness of the diagonal member
"""
function diagonal_bar_bearing_stress(force, diameter, thickness)
    return (force) / (2 * diameter * thickness)
end

"""
cross_bar_bearing_stress(force, diameter, thickness), FOR SQUARE CROSSBAR
This function returns the bearing stress in the crossbar for a square shape

force is the force in the crossbar
diameter is the diameter of the hole in the crossbar
thickness is the thickness of the crossbar
"""
function cross_bar_bearing_stress(force, diameter, thickness)
    return (force) / (2 * diameter * thickness)
end

"""
cross_bar_bearing_stress_round(force, diameter, diameter_crossbar), FOR ROUND CROSSBAR
This function returns the bearing stress in the crossbar for a solid round

force is the force in the crossbar
diameter is the diameter of the hole in the crossbar
diameter_crossbar is the diameter of the crossbar
"""
function cross_bar_bearing_stress(force, diameter, diameter_crossbar)
    return (force) / (diameter * diameter_crossbar)
end

"""
pin_bearing_stress(force, diameter, thickness), FOR SQUARE CROSSBAR
This function returns the bearing stress in the pin for a square shape crossbar

force is the force in the crossbar
diameter is the diameter of the pin
thickness is the thickness of the crossbar
"""
function pin_bearing_stress(force, diameter, thickness)
    return (force) / (2 * diameter * thickness)
end

"""
pin_bearing_stress(force, diameter, diameter_crossbar), FOR ROUND CROSSBAR
This function returns the bearing stress in the pin for a solid round crossbar

force is the force in the crossbar
diameter is the diameter of the pin
diameter_crossbar is the diameter of the crossbar
"""
function pin_bearing_stress(force, diameter, diameter_crossbar)
    return (force) / (diameter * diameter_crossbar)
end

"""
diagonal_bar_tearout_stress(force, length, thickness)
This function returns the tearout stress in the diagonal bar

force is the force in the diagonal member
length is the distance from the center of the hole to the edge of the diagonal member
thickness is the thickness of the diagonal member
"""
function diagonal_bar_tearout_stress(force, length, thickness)
    return (force) / (4 * length * thickness)
end

"""
diagonal_bar_tensile_stress(force, width, diameter, thickness)
This function returns the tensile stress in the diagonal bar

force is the force in the diagonal member
width is the width of the diagonal member
diameter is the diameter of the hole in the diagonal member
thickness is the thickness of the diagonal member
"""
function diagonal_bar_tensile_stress(force, width, diameter, thickness)
    return (force) / ((width - diameter) * thickness * 2)
end

"""
pin_direct_shear(force, n, diameter)
This function returns the direct shear force in the pin

force is the force in the cross bar
n is the number of shear planes in the pin
diameter is the diameter of the pin
"""
function pin_direct_shear(force, n, diameter)
    return (force) / (n * (pi / 4) * (diameter ^ 2))
end

"""
von_mises_stress(sigma_x, sigma_y, tau_xy)
This function returns the von-mises stress from plane stress

sigma_x is the stress in the x direction
sigma_y is the stress in the y direction
tau_xy is the shear in the xy direction
"""
function von_mises_stress(sigma_x, sigma_y, tau_xy)
    return sqrt((sigma_x^2) + (sigma_y^2) - (sigma_x*sigma_y) + (3*(tau_xy^2)))
end

#############################################################################
# Buckling functions:
#############################################################################

function k(I, A)
    return sqrt(I / A)
end

function l_over_k_1(c, E, s_y)
    return sqrt((2*(pi^2)*c*E) / (s_y))
end

function s_r(l, k)
    return l / k
end

"""
max_load_buckling(c, E, s_y, I, A, l)
This function returns the critical loading force for an Euler or Johnson column

c is the end-condition coefficient
E is the modulus of Elasticity
s_y is the yield strength
I is the moment of inertia
A is the area
l is the length
"""
function max_load_buckling(c, E, s_y, I, A, l)
    K = k(I, A)
    lk1 = l_over_k_1(c, E, s_y)
    S_r = s_r(l, K)
    if S_r > lk1
        return (A * c * (pi^2) * E) / (S_r ^ 2)
    else
        return A * (s_y - (1 / (c * E)) * ((s_y * S_r) / (2 * pi))^2)
    end
end

max_load_buckling

## Tearout on Diagonal Member

### Failure Criteria: Von-Mises
### Strength Value Used: $S_{y,d} = 16000$ psi
### Stress Equation: $\sigma_{tearout, d} = \frac{F_d}{4*l*t}$

In [14]:
function safety_diagonal_tear(applied_force)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = diagonal_bar_tearout_stress(F_d, l, t_d)
    sigma_von = von_mises_stress(0.0, 0.0, sigma_nom)
    n = S_y_d / sigma_von
    return n
end
find_zero(force -> safety_diagonal_tear(force) - 1, 700.0)

2375.8406309295137

### Applied Load Predicted to Cause Failure: $2375.84$ lbf

## Axial on Diagonal Member 

### Failure Criteria: Von-Mises
### Strength Value Used: $S_{y,d} = 16000$ psi
### Stress Equation: $\sigma_{axial,d} = \frac{F_d}{2*t_d*(w_d-d)}$

In [15]:
function safety_diagonal_axial(applied_force)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = diagonal_bar_tensile_stress(F_d, w_d, d, t_d)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_d / sigma_von
    return n
end
find_zero(force -> safety_diagonal_axial(force) - 1, 700.0)

5143.845854320518

### Applied Load Predicted to Cause Failure: $5143.85$ lbf

## Bearing Stress on Diagonal Member

### Failure Criteria: Von-Mises
### Strength Value Used: $S_{y,d} = 16000$ psi
### Stress Equation: $\sigma_{bearing,d} = \frac{F_d}{2*t_d*d}$

In [16]:
function safety_diagonal_bearing(applied_force)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = diagonal_bar_bearing_stress(F_d, d, t_d)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_d / sigma_von
    return n
end
find_zero(force -> safety_diagonal_bearing(force) - 1, 700.0)

1285.9614635801295

### Applied Load Predicted to Cause Failure: $1285.96$ lbf

# Bearing Stress on Crossbar

### Failure Criteria: Von-Mises
### Strength Value Used: $S_{y,d} = 32000$ psi
### Stress Equation: $\sigma_{bearing,cb} = \frac{F_{cb}}{2*t_{cb}*d}$

In [17]:
function safety_crossbar_bearing(applied_force)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = cross_bar_bearing_stress(F_cb, d, t_cb)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_cb / sigma_von
    return n
end
find_zero(force -> safety_crossbar_bearing(force) - 1, 700.0)

402.9740991864001

### Applied Load Predicted to Cause Failure: $805.94$ lbf

## Buckling on Crossbar

### Failure Criteria: Euler/Johnson (Johnson was used this particular time, but the function I have will use the most appropriate one)
### Strength Value Used: $S_{y,cb} = 32000$ psi, $E_{cb} = 30000000$ psi
### Stress Equation: 
### Johnson: $P_{cr} = A[S_{yc} - (\frac{1}{CE})(\frac{S_{yc}S_r}{2\pi})^2]$
### Euler: $P_{cr} = AC\frac{\pi^2E}{S_r^2}$

In [18]:
function safety_buckling_crossbar(applied_force) #for square cross section crossbar
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    A = (w_cb ^ 2) - (w_cb - (2 * t_cb)) ^ 2
    I = ((w_cb ^ 4 / 12)) - ((w_cb - (2 * t_cb)) ^ 4 / 12)
    max_buck = max_load_buckling(C, E_cb, S_y_cb, I, A, l_cb)
    n = max_buck / F_cb
    return n
end
find_zero(force -> safety_buckling_crossbar(force) - 1, 700.0)

2441.584613881853

In [20]:
function safety_buckling_crossbar_round(applied_force) #for solid round cross section crossbar
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    A = ((d_cb / 2) ^ 2) * pi
    I = (pi / 64) * ((d_cb) ^ 4)
    max_buck = max_load_buckling(C, E_cb, S_y_cb, I, A, l_cb)
    n = max_buck / F_cb
    return n
end
find_zero(force -> safety_buckling_crossbar_round(force) - 1, 700.0)

3728.6437688087767

### Applied Load Predicted to Cause Failure: $2441.58$ lbf

## Shear on Pin

### Failure Criteria: Von Mises
### Strength Value Used: $S_{y,p} = 140000$ psi
### Stress Equation: $\tau_{direct,p} = \frac{F_{cb}}{N*(\pi/4)*d^2}$

In [8]:
function safety_pin_shear(applied_force)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    tau_nom = pin_direct_shear(F_cb, N, d)
    sigma_von = von_mises_stress(0.0, 0.0, tau_nom)
    n = S_y_p / sigma_von
    return n
end
find_zero(force -> safety_pin_shear(force) - 1, 700.0)

6661.978065559953

### Applied Load Predicted to Cause Failure: $6661.98$ lbf

## Bearing Stress on Pin

### Failure Criteria: Von-Mises
### Strength Value Used: $S_{y,p} = 140000$ psi
### Stress Equation: $\sigma_{bearing,p} = \frac{F_{cb}}{2*d*t_{cb}}$

In [9]:
function safety_pin_shear(applied_force)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = pin_bearing_stress(F_cb, d, t_cb)
    tau_nom = pin_direct_shear(F_cb, N, d)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_p / sigma_von
    return n
end
find_zero(force -> safety_pin_shear(force) - 1, 700.0)

3526.0233678810005

### Applied Load Predicted to Cause Failure: $3526.02$ lbf


| Analysis Number | Location of Failure | Failure Mode | Strength Value Used | Stress Equation | Applied Load Predicted to Cause Failure (lbf) |
|------|------|------|------|------|------|
|  1   | Diagonal Member | Tearout | $S_{y,d} = 16000$ psi | $\sigma_{tearout, d} = \frac{F_d}{4*l*t}$ | 2375.84 |
|  2   | Diagonal Member | Axial | $S_{y,d} = 16000$ psi   | $\sigma_{axial,d} = \frac{F_d}{2*t_d*(w_d-d)}$ | 5143.85 |
|  3   | Diagonal Member | Bearing Stress | $S_{y,d} = 16000$ psi   | $\sigma_{bearing,d} = \frac{F_d}{2*t_d*d}$ | 1285.96 |
|  4   | Crossbar | Bearing Stress | $S_{y,cb} = 32000$ psi   | $\sigma_{bearing,cb} = \frac{F_{cb}}{2*t_{cb}*d}$ | 805.94 |
|  5   | Crossbar | Buckling | $S_{y,cb} = 32000$ psi | Johnson: $P_{cr} = A[S_{yc} - (\frac{1}{CE})(\frac{S_{yc}S_r}{2\pi})^2]$ Euler: $P_{cr} = AC\frac{\pi^2E}{S_r^2}$ | 2441.58 |
|  6   | Pin | Shear | $S_{y,p} = 140000$ psi | $\tau_{direct,p} = \frac{F_{cb}}{N*(\pi/4)*d^2}$ | 6661.98 |
|  7   | Pin | Bearing Stress | $S_{y,p} = 140000$ psi   | $\sigma_{bearing,p} = \frac{F_{cb}}{2*d*t_{cb}}$ | 3526.02 |


## a. What values in the experiments are most comparable to the model predictions?

Our model predictions predicted tearout yielding at about 2375 lbf and buckling at about 2440 lbf, which were the lowest two points of failure... if you ignore the fact that bearing stress on both the diagonal member and the crossbar was supposed to have yielding at about a third to half of that. Setting aside the bearing, our model predictions line up quite closely to the yield loads in all three tests. (We are expecting the material properties we used to be underestimates, such that the real material will slightly outpreform the model.) 

If we look at all three of these failure modes together, they all show up in the images. And perhaps the reason that bearing failure wasn't listed as a failure mode was because it progressed to the part where it became tearout.

## b. What are the key reasons for differences between the models and the experiments? Be sure to address issues around all aspects of the experiments (materials, dimensions, etc.) and the calculations (assumptions).

Probably the most glaring difference between the experimental results and the model is the fact that bearing failure wasn't the cause of failure every time as it has by far the lowest load for failure. What makes sense to me is that some yielding in the holes on the diagonal members was not sufficient to cause catastrophic failure, and as a result, it didn't show in the experimental results. (Although you can see plastic deformation around the holes on the diagonal members.) 

In brief, here are some discrepancies between the model and the real experiment:
1) Material properties. The advertized material properties likely represent a lower threshold for the actual material properties. This helps explain why the yielding loads were close, but larger than the predicted yielding load.

2) Geometry differences. In the model we assumed things like the bearing area was the projected area of the inside of the hole or assumed that the shear was similar to that caused by a pin holding together just two sheets of metal. This last one is rather different from our situation where we had (on one half of the pin three sheets of metal being "pulled" in three different angles.)

3) The end condition coefficient for buckling is also a rather inprecise number. Using 1 gives us a decent model, but it even varies on the chart provided to us in class depending on how risky you want to be in your design.

4) Another important consideration is that the mathematical model assumes a perfect symmetrical loading, while that is not necessarily the case in the experiments.

## c. Which 1-2 issues are likely to be most significant sources of differences? Justify your explanation.

I would say that the largest issue is the fact that yielding due to bearing doesn't appear. I would expct to see it, if not the cause of catastrophic failure, resulting in a much lower yield load. I can't think of any significant assumptions which would so drastically change the value of load under which yielding due to bearing would occur. From the pictures, it looks like it did occur, but it did not cause a great change in the displacement vs load figure. 

Other than that, I would make the case that the material values used for the material properties is the single greatest cause for any differences. However, it is wise for a company to advertise their material properties as being on the low end of the actual range for their product. 

## d. Based on these observations with a ductile material, What level of accuracy would you expect from the hand calculations? (Express as a percentage). Express any assumptions behind your answer.

I am going to assume that there was bearing failure in the holes of the diagonal members more or less where we expected, but that it doesn't show up in the data. 

With that assumption, we expected failure due to yielding at 2375 lbf. Comparing this to the average of the three tests, we have an error of about 9.92% of the real yield load when predicting the yield load. Considering the fact that we want to operate with a safety factor greater than 1, this is acceptable.

In [14]:
function fatigue(d)
    sigma = (138528)*(d^-3)*(1/(1+(-0.304521/(sqrt(d)))))
    design_factor = 1.6
    b = (-1/3)*log10(98.4/(39.0276*(d^-0.107)))
    a = (98.4)/(1000^b)
    N = 10*950*60
    return (sigma*design_factor)-a*(N^b)
end
find_zero(fatigue, 0.5)

19.643120369710566